In [59]:
import os
import pandas as pd
import json
import re

In [12]:
page_data = pd.read_csv("../raw_data/page.csv")

In [124]:
def write_batch_to_csv(batch_data, output_dir, batch_number):
    # Erstelle den Dateinamen basierend auf der Batch-Nummer
    output_file_path = f"{output_dir}/batch_{batch_number}.csv"
    # Konvertiere die Batch-Daten zu einem Pandas DataFrame und speichere sie als CSV
    df = pd.DataFrame(batch_data)
    df.to_csv(output_file_path, index=False, encoding='utf-8')


def create_nodes(data, page_data):
    node_list = []
    relation_list = []

    current_page_title = page_data[page_data["page_id"]==data["page_id"]].title.iloc[0]
    current_page_item_id = page_data[page_data["page_id"]==data["page_id"]].item_id.iloc[0]

    for i, section in enumerate(data["sections"]):
        if section["name"] not in ["Introduction","See also", "External links"]:

            item = {"title": current_page_title + " - " +  section["name"],
                    "text": section["text"],
                    "page_id": data["page_id"],
                    "item_id_parent": current_page_item_id,
                    "item_id": str(current_page_item_id) + "_" + str(i)} # Mapping über item_id und section type
            node_list.append(item)
            relation_list.append({"source_item_id": current_page_item_id, "target_item_id": item["item_id"]})

    return node_list, relation_list


def convert_jsonl_to_csv_batches(jsonl_file_path, output_dir, pages, batch_size=10):
    # Zähler, um die aktuelle Batch-Nummer zu verfolgen
    batch_number = 0
    batch_data = []

    # Öffne die JSONL-Datei zeilenweise
    with open(jsonl_file_path, 'r', encoding='utf-8') as jsonl_file:
        for line in jsonl_file:
            # JSON-Zeile parsen und zur Batch-Liste hinzufügen

            line = json.loads(line.strip())
            nodes, relations = create_nodes(line, pages)
            batch_data += nodes
            
            # Wenn die Batch-Größe erreicht wurde, schreibe die Daten ins CSV
            if len(batch_data) == batch_size:
                write_batch_to_csv(batch_data, output_dir, batch_number)
                break
                batch_number += 1
                batch_data = []  # Leere die Liste, um den Speicher zu entlasten

        # Verarbeite den verbleibenden Rest (falls noch Daten vorhanden sind)
        if batch_data:
            write_batch_to_csv(batch_data, output_dir, batch_number)

## Irgendwas stimmmt nicht mit der Länge des Batches...


In [125]:
jsonl_file_path = '../raw_data/link_annotated_text.jsonl'
output_dir = '../neo4j_db/import/sections'
convert_jsonl_to_csv_batches(jsonl_file_path, output_dir, page_data)

KeyboardInterrupt: 

## Hier weiter!

In [16]:
page_data[page_data["page_id"]==12].title.loc[0]

'Anarchism'

In [110]:
len(page_data)

5362174

# ZU klären: Soll die Introduction an den parent node angehangen werden?

In [108]:
def create_dataframe(data, page_data):
    node_list = []
    relation_list = []

    for item in data:
        current_page_title = page_data[page_data["page_id"]==item["page_id"]].title.iloc[0]
        current_page_item_id = page_data[page_data["page_id"]==item["page_id"]].item_id.iloc[0]

        for i, section in enumerate(item["sections"]):
            if section["name"] not in ["Introduction","See also", "External links"]:

                item = {"title": current_page_title + " - " +  section["name"],
                        "text": section["text"],
                        "page_id": item["page_id"],
                        "item_id_parent": current_page_item_id,
                        "item_id": str(current_page_item_id) + "_" + str(i)} # Mapping über item_id und section type
                node_list.append(item)
                relation_list.append({"source_item_id": current_page_item_id, "target_item_id": item["item_id"]})

    return pd.DataFrame(node_list), pd.DataFrame(relation_list)

In [52]:
page_data[page_data["page_id"]==25].title.iloc[0]

'Autism'

In [109]:
df1, df2 =create_dataframe(data, page_data)

12
25
39
290
303
305
307


In [105]:
iter = re.finditer("\s*===[A-Za-z\s]*===", data[6]["sections"][1]["text"])

In [106]:
for i in iter:
    print(i.start())

0


In [107]:
data[6]["sections"][1]["text"]

'===Early life=== Abraham Lincoln was born on February 12, 1809, as the second child of Thomas Lincoln and Nancy Hanks Lincoln, in a one-room log cabin on Sinking Spring Farm near Hodgenville, Kentucky. He was a descendant of Samuel Lincoln, an Englishman who migrated from Hingham, Norfolk, to its namesake, Hingham, Massachusetts, in 1638. Samuel\'s grandson and great-grandson began the family\'s westward migration, passing through New Jersey, Pennsylvania, and Virginia. Lincoln\'s paternal grandfather and namesake, Captain Abraham Lincoln, moved the family from Virginia to Jefferson County, Kentucky, in the 1780s. Captain Lincoln was killed in an Indian raid in 1786. His children, including eight-year-old Thomas, Abraham\'s father, witnessed the attack. Thomas then worked at odd jobs in Kentucky and in Tennessee, before settling with members of his family in Hardin County, Kentucky, in the early 1800s. . Lincoln\'s mother, Nancy, is widely assumed to have been the daughter of Lucy Han

# Hier könnten noch weitere subnodes entstehen durch die === Marker

Datenqualität ist aber bescheiden

In [56]:
df1

,title,text,page_id,item_id_parent,item_id
0,"Anarchism - Etymology, terminology, and defini...",The etymological origin of the word anarchism ...,12,6199,6199_1
1,Anarchism - History,"During the prehistoric era of mankind, an esta...",12,6199,6199_2
2,Anarchism - Anarchist schools of thought,Anarchist schools of thought had been generall...,12,6199,6199_3
3,Anarchism - Internal issues and debates,As anarchism is a philosophy that embodies man...,12,6199,6199_4
4,Anarchism - Topics of interest,Intersecting and overlapping between various s...,12,6199,6199_5
5,Anarchism - Criticism{{anchor|Criticisms}},Moral and pragmatic criticism of anarchism inc...,12,6199,6199_6
6,Anarchism - List of anarchist societies,* Federation of Neighborhood Councils-El Alto ...,12,6199,6199_7
7,Autism - Characteristics,"Autism is a highly variable, neurodevelopmenta...",25,38404,38404_1
8,Autism - Causes,It has long been presumed that there is a comm...,25,38404,38404_2
9,Autism - Mechanism,Autism's symptoms result from maturation-relat...,25,38404,38404_3


In [9]:
data

[{'page_id': 12,
  'sections': [{'name': 'Introduction',
    'text': "Anarchism is an anti-authoritarian political and social philosophy that rejects hierarchies deemed unjust and advocates their replacement with self-managed, self-governed societies based on voluntary, cooperative institutions. These institutions are often described as stateless societies, although several authors have defined them more specifically as distinct institutions based on non-hierarchical or free associations. Anarchism's central disagreement with other ideologies is that it holds the state to be undesirable, unnecessary, and harmful. Anarchism is usually placed on the far-left of the political spectrum, and much of its economics and legal philosophy reflect anti-authoritarian interpretations of communism, collectivism, syndicalism, mutualism, or participatory economics. As anarchism does not offer a fixed body of doctrine from a single particular worldview, many anarchist types and traditions exist and var

### Remove See also and External Links -> Even though they might contain some links

In [11]:
data[0]

{'page_id': 12,
 'sections': [{'name': 'Introduction',
   'text': "Anarchism is an anti-authoritarian political and social philosophy that rejects hierarchies deemed unjust and advocates their replacement with self-managed, self-governed societies based on voluntary, cooperative institutions. These institutions are often described as stateless societies, although several authors have defined them more specifically as distinct institutions based on non-hierarchical or free associations. Anarchism's central disagreement with other ideologies is that it holds the state to be undesirable, unnecessary, and harmful. Anarchism is usually placed on the far-left of the political spectrum, and much of its economics and legal philosophy reflect anti-authoritarian interpretations of communism, collectivism, syndicalism, mutualism, or participatory economics. As anarchism does not offer a fixed body of doctrine from a single particular worldview, many anarchist types and traditions exist and variet